## Basic Statistics on UKB dataset

Set up global variables and import modules

In [ ]:
import os
import sys
import pandas as pd
from matplotlib import pyplot as plt

ukb_root = '/project_freenas/3022017.02/UKB'
sys.path.append(os.path.join(ukb_root,'scripts'))
from ukb_utils import get_variables_UKB, lookup_UKB

ukb_idp_dir = os.path.join(ukb_root,'phenotypes','current')

UKB field code and names dictionary

Read dataframe and check field codes

In [64]:
basic_demo = pd.read_csv(os.path.join(ukb_idp_dir,'01_basic_demographics.csv'))
fields_ID = basic_demo.columns

Lookup field IDs with Lookup_UKB function

In [101]:
field_list = pd.DataFrame(columns=['Field', 'FieldID'])
for idf in fields_ID[1:]:
    fld = lookup_UKB(field_ids=[idf]) 
    field_list = field_list.append(fld)
field_list.head(n=3)

,Field,FieldID
7,Sex,31
9,Year of birth,34
1578,Year of birth,22200
